In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt



def vol(stock, second_interval):
    log_r1 = []
    time_IDs = np.unique(stock.iloc[:, 0])
    print(time_IDs)

    #Generate Weighted Average Price
    stock["WAP"] = (stock["bid_price1"] * stock["ask_size1"] + stock["ask_price1"] * stock["bid_size1"]) / (stock["bid_size1"] + stock["ask_size1"])

    for i in range(len(time_IDs)):
        sec = stock.loc[stock.iloc[:, 0] == time_IDs[i], 'seconds_in_bucket'].values
        price = stock.loc[stock.iloc[:, 0] == time_IDs[i], 'WAP'].values
        log_r = np.log(price[1:] / price[0:(len(price) - 1)])
        log_r1.append(pd.DataFrame({'time': sec[1:], 'log_return': log_r}))
        time_no_change = np.setdiff1d(np.arange(1, 601), log_r1[i]['time'].values)
        if len(time_no_change) > 0:
            new_df = pd.DataFrame({'time': time_no_change, 'log_return': 0})
            log_r1[i] = pd.concat([log_r1[i], new_df])
            log_r1[i] = log_r1[i].sort_values(by='time')

    vol = []
    def comp_vol(x):
        return np.sqrt(np.sum(x ** 2))

    for i in range(len(log_r1)):
        log_r1[i]['time_bucket'] = np.ceil(log_r1[i]['time'] / second_interval)
        vol.append(log_r1[i].groupby('time_bucket')['log_return'].agg(comp_vol).reset_index())
        vol[i].columns = ['time_bucket', 'volatility']
    out = []
    times = []
    for i in range(len(vol)):
        for j in range(int(600/second_interval)):
            out.append(vol[i]["volatility"][j])
            times.append(int(vol[i]["time_bucket"][j]))
    vol = np.array(out)

    return vol, times


TRYING TO FIND OPTIMAL ALPHA VALUE

In [32]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
train_volatility, test_volatility = train_test_split(volatility, test_size=0.2)


# Set the range of alpha values to test
alphas = np.arange(0.01, 1, 0.01)
print(alphas)

# Initialize variables to store the best alpha and corresponding performance metric
best_alpha = 0
best_mse = float('inf')

# Loop through alpha values
for alpha in alphas:
    # Calculate the EWMA of the training set using pandas
    ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility),alpha=alpha).mean()
    # Initialize list for predicted volatilities
    predicted_volatility = []
    # Loop through the test set
    for i in range(len(test_volatility)):
        # Add current test point to training data
        train_volatility = np.append(train_volatility, test_volatility[i])
        # Update EWMA with new point
        ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility), alpha=alpha).mean()
        # Get the last value of the EWMA as the forecasted volatility
        forecasted_volatility = ewma_train_volatility.values[-1]
        # Append forecasted value to list of predicted volatilities
        predicted_volatility.append(forecasted_volatility)
    # Calculate mean squared error between predicted and actual volatilities
    mse = mean_squared_error(test_volatility, predicted_volatility)
    # Update best alpha and corresponding performance metric if applicable
    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha

# Print the optimal alpha and corresponding performance metric
print(f"Optimal alpha: {best_alpha}")
print(f"Best MSE: {best_mse}")


# Print optimal alpha value
print('Optimal alpha value:', alpha)


[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


DataError: No numeric types to aggregate

In [12]:

def ewma_mse(volatility, span):
    # Split the data into training and testing sets
    #train_volatility, test_volatility = train_test_split(volatility, test_size=0.2, random_state= 1)
    train_volatility = volatility[:round(len(volatility)*0.8)]
    test_volatility = volatility[round(len(volatility)*0.8):]


    # Calculate the EWMA of the training set using pandas
    ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility), span=span).mean()

    # Initialize list for predicted volatilities
    predicted_volatility = []

    # Loop through the test set
    for i in range(len(test_volatility)):
        # Add current test point to training data
        train_volatility = np.append(train_volatility, test_volatility[i])
        # Update EWMA with new point
        ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility), span=span).mean()
        # Get the last value of the EWMA as the forecasted volatility
        forecasted_volatility = ewma_train_volatility.values[-1]
        # Append forecasted value to list of predicted volatilities
        predicted_volatility.append(forecasted_volatility)

    pd.DataFrame(test_volatility,predicted_volatility)

    return predicted_volatility


def show_plot(test_volatility, predicted_volatility):
    # Plot predicted values vs actual test values
    plt.plot(range(len(test_volatility)), test_volatility, label='Actual Volatility')
    plt.plot(range(len(predicted_volatility)), predicted_volatility, label='Predicted Volatility')
    plt.legend()
    plt.title('Actual vs Predicted Volatility')
    plt.xlabel('Time')
    plt.ylabel('Volatility')
    plt.show()

    # Calculate mean squared error (MSE)
    mse = np.mean((predicted_volatility - test_volatility)**2)
    print("Mean Squared Error (MSE):", mse)

    # Calculate root mean squared error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
def EWMA_each_stock(df,span=3):

    # Read the CSV file into a pandas DataFrame
    stock = pd.read_csv(stock)

    # Get a list of unique time ids
    time_ids = stock['time_id'].unique()

    # Loop through the unique time ids
    ls = []
    first = True
    for time_id in time_ids:
        
        time_id_rows = stock[stock['time_id'] == time_id]
        ls.append(ewma_mse(df["volatility"],span))


df = pd.DataFrame({"time_id":[1, 1, 2, 2],
                    "volatility":[5, 6, 7, 8]})
  
EWMA_each_stock(df,3)

In [79]:
def ewma_predict(df, span):
    # Split the data into training and testing sets

    volatility = list(df["volatility"])
    ans = df.tail(round(len(volatility)*0.2))

    train_volatility = volatility[:round(len(volatility)*0.8)]
    test_volatility = volatility[round(len(volatility)*0.8):]
    # Calculate the EWMA of the training set using pandas
    ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility), span=span).mean()

    # Initialize list for predicted volatilities
    predicted_volatility = []

    # Loop through the test set
    for i in range(len(test_volatility)):
        
        # Add current test point to training data
        train_volatility = np.append(train_volatility, test_volatility[i])
        # Update EWMA with new point
        ewma_train_volatility = pd.DataFrame.ewm(pd.Series(train_volatility), span=span).mean()
        # Get the last value of the EWMA as the forecasted volatility
        forecasted_volatility = ewma_train_volatility.values[-1]
        # Append forecasted value to list of predicted volatilities
        predicted_volatility.append(forecasted_volatility)
    
    ans["predicted"] = predicted_volatility
    return ans


def EWMA_each_stock(stock,span=3):
    # Get a list of unique time ids
    time_ids = stock['time_id'].unique()
    # Loop through the unique time ids
    first = True
    for time_id in time_ids:

        time_id_rows = stock[stock['time_id'] == time_id]

        if first == True:
            ans = ewma_predict(time_id_rows,span)
            print(ans)
            first = False
        else:
            ans = ans.append(ewma_predict(time_id_rows,span))
            print(ans)

    return ans

df = pd.DataFrame({"time_id":[1, 1, 1,1, 1, 1,1,1,1,1,2,2,2,2,2,2,2],
                   "seconds":[1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
                    "volatility":[5, 6, 7, 8,9,10,11,12,13,14,15,16,17,18,19,20,21]})



EWMA_each_stock(df)


   time_id  seconds  volatility  predicted
8        1        9          13  12.017613
9        1       10          14  13.009775
    time_id  seconds  volatility  predicted
8         1        9          13  12.017613
9         1       10          14  13.009775
16        2       17          21  20.055118


C:\Users\cecil\AppData\Local\Temp\ipykernel_12856\1767191143.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans["predicted"] = predicted_volatility
C:\Users\cecil\AppData\Local\Temp\ipykernel_12856\1767191143.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans["predicted"] = predicted_volatility
C:\Users\cecil\AppData\Local\Temp\ipykernel_12856\1767191143.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a

,time_id,seconds,volatility,predicted
8,1,9,13,12.017613
9,1,10,14,13.009775
16,2,17,21,20.055118
